In [ ]:
import sys
sys.path.append(".")
sys.path.append("noisepy4das_repo/NoisePy4DAS-SeaDAS/src")
sys.path.append("noisepy4das_repo/NoisePy4DAS-SeaDAS/DASstore")
import cv2
import os
import gc
import h5py
import math
import time
import DAS_module
import numpy as np
import pandas as pd
import matplotlib
import scipy.signal as sgn

from tqdm import tqdm
from obspy import UTCDateTime
from datetime import datetime
from datetime import timedelta
from functools import partial
from scipy.signal import butter
from scipy.signal import detrend
from scipy.signal import decimate
from scipy.signal import filtfilt
from scipy.signal import spectrogram
from scipy.interpolate import interp1d
from dasstore.zarr import Client
from multiprocessing import Pool
from matplotlib import pyplot as plt
from das_util import read_decimate, get_tstamp, calc_NFFT
from das_util import next_power_of_2

%matplotlib inline
matplotlib.rcParams.update({'font.size': 16})
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

In [ ]:
def fk_filter_2cones(vsp, w1=0, w2=0, cone1=False, cone2=False):
    n1, n2 = vsp.shape
    nf = next_power_of_2(n1)
    nk = next_power_of_2(n2)

    nf2=int(nf/2)
    nk2=int(nk/2)
    
    fk2d = np.fft.fft2(vsp, s=(nf,nk))
    fk2d = np.fft.fftshift(fk2d, axes=(-2,-1))
    
    nw1 = int(np.ceil(w1*nk))
    nw2 = int(np.ceil(w2*nf))

    mask1=np.ones((nf,nk), dtype=np.float64)
    mask2=np.ones((nf,nk), dtype=np.float64)

    if cone1:
        for j in np.arange(nk2-nw1, nk2+1):
            th1 = int((j-nk2+nw1) * nf2/nw1)

            mask1[:th1, j] = 0
            mask1[nf-th1:, j] = 0
            mask1[:th1, nk-j] = 0
            mask1[nf-th1:, nk-j] = 0

    if cone2:
        for j in np.arange(0, nk2):
            th2 = int(nf2 - (nw2/nk2)*(nk2-j))
            mask2[th2:nf-th2+1, j] = 0
            if j != 0:
                mask2[th2:nf-th2+1, nk-j] = 0


    mask = mask2*mask1
    
    filtered_2d = fk2d * mask
    tmp = np.fft.ifftshift(filtered_2d)
    output = np.fft.ifft2(tmp, s=(nk,nf), axes=(-1, -2))
    
    return output[:n1,:n2], filtered_2d, fk2d

In [ ]:
def stretch(wave1, wave2, time, maxshift=0, max_ratio=2):

    interp_f = interp1d(time, wave2, bounds_error=False, fill_value=0.)
    n1 = np.sum(np.square(wave1))
    dt = time[1] - time[0]
    cc = 0
    relative_ratio = 1
    npts = len(time)

    for ratio in np.arange(1/max_ratio, max_ratio, 0.01):
        dt_new = dt / ratio
        time_new = np.arange(time[0], time[-1], dt_new)
        wave_new = interp_f(time_new)
        
        n2 = np.sum(np.square(wave_new))
        corr = sgn.correlate(wave1, wave_new) / np.sqrt(n1 * n2)

        l_maxshift = min(len(wave_new), maxshift)
        r_maxshift = min(len(wave1), maxshift)

        st_pt = len(wave_new) - l_maxshift
        en_pt = len(wave_new) + r_maxshift+1

        cc_best = np.nanmax(corr[st_pt: en_pt])

        if cc < cc_best:
            cc = cc_best
            relative_ratio = ratio

    dt_new = dt / relative_ratio
    time_new = np.arange(time[0], time[-1], dt_new)
    wave_new = interp_f(time_new)
    
    
    return wave_new, np.arange(len(wave_new))*dt, relative_ratio, cc

In [ ]:
def stretch_distribution(wave1, wave2, time, maxshift=0, max_ratio=2):

    stretch_range = np.arange(1/max_ratio, max_ratio, 0.01)
    
    interp_f = interp1d(time, wave2, bounds_error=False, fill_value=0.)
    n1 = np.sum(np.square(wave1))
    dt = time[1] - time[0]
    cc = np.zeros(len(stretch_range), dtype = np.float32)
    npts = len(time)

    for i, ratio in enumerate(stretch_range):
        dt_new = dt / ratio
        time_new = np.arange(time[0], time[-1], dt_new)
        wave_new = interp_f(time_new)
        
        n2 = np.sum(np.square(wave_new))
        corr = sgn.correlate(wave1, wave_new) / np.sqrt(n1 * n2)

        l_maxshift = min(len(wave_new), maxshift)
        r_maxshift = min(len(wave1), maxshift)

        st_pt = len(wave_new) - l_maxshift
        en_pt = len(wave_new) + r_maxshift+1

        cc[i] = np.nanmax(corr[st_pt: en_pt])
    
    return stretch_range, cc

In [ ]:
data_dir = '/1-fnp/petasaur/p-wd05/harper_plots'
file_list = np.array(os.listdir(data_dir))

acqu_time = np.array([get_tstamp(i) for i in file_list])
new_index = np.argsort(np.array(acqu_time)-acqu_time[0])
acqu_time = acqu_time[new_index]

file_list = file_list[new_index]
file_path = [os.path.join(data_dir,i) for i in file_list]

# %% reasonable acquisition time period
list_all = np.arange(28,2468)
num_sample_all = np.zeros(len(list_all), dtype=np.float64)
sample_rate_all = np.zeros(len(list_all), dtype=np.float64)
delta_space_all = np.zeros(len(list_all), dtype=np.float64)
for i,j in enumerate(list_all):
    with h5py.File(file_path[j],'r') as f:      
        num_sample_all[i]  = len(f['Acquisition']['Raw[0]']['RawDataTime'][:])
        sample_rate_all[i] = f['Acquisition']['Raw[0]'].attrs['OutputDataRate']
        delta_space_all[i] = f['Acquisition'].attrs['SpatialSamplingInterval']
        
# %% exclude files that dropped samples
ind_good = np.where(num_sample_all == 120000)[0]
list_all = list_all[ind_good]
delta_space = delta_space_all[ind_good][0]
num_sample = num_sample_all[ind_good][0]
sample_rate = sample_rate_all[ind_good][0]
delta_time = 1.0 / sample_rate

print(f'good acqusition for {len(ind_good)} minutes')

del num_sample_all, file_path
gc.collect()
# %% See if the acquisition time is continuous
file_list = file_list[list_all]
acqu_time = acqu_time[list_all]
file_path = [os.path.join(data_dir,i) for i in file_list]
plt.close('all')
fig, ax = plt.subplots(1, 1, figsize=(13, 2.5), constrained_layout=True)
ax.scatter(list_all, acqu_time.astype('datetime64[m]'), marker='o', s=0.1, edgecolors='k')

In [ ]:
start_file = np.arange(0, len(acqu_time), 5)
time_stamps = acqu_time[start_file]

samp_freq = 500                # targeted sampling rate
freq_norm   = 'no'             # 'no' for no whitening, or 'rma' for running-mean average, 'phase_only' for sign-bit normalization in freq domain.
time_norm   = 'one_bit'        # 'no' for no normalization, or 'rma', 'one_bit' for normalization in time domain
cc_method   = 'xcorr'          # 'xcorr' for pure cross correlation, 'deconv' for deconvolution; FOR "COHERENCY" PLEASE set freq_norm to "rma", time_norm to "no" and cc_method to "xcorr"
smooth_N    = 50               # moving window length for time domain normalization if selected (points)
smoothspect_N  = 50            # moving window length to smooth spectrum amplitude (points)
maxlag      = 5                # lags of cross-correlation to save (sec)

# criteria for data selection
max_over_std = 10 *9              # threshold to remove window of bad signals: set it to 10*9 if prefer not to remove them
num_sample = 30000
cc_len = delta_time * num_sample  # correlate length in second
step   = delta_time * num_sample  # stepping length in second

n_pair = 1
n_lag = maxlag * samp_freq * 2 + 1

In [ ]:
with h5py.File('/fd1/QibinShi_data/England_farm/psd_all_channel.hdf5', 'r') as f:
    PSD_all_time = f['psd_all_time'][:]
    freq = f['frequency'][:]

In [ ]:
hourly_index=[67,127,187,247,299,359,419,466,526,586,646,706,766,826,886,946,1003,1063,1123,
         1183,1243,1302,1362,1422,1482,1542,1599,1659,1719,1779,1839,1899,1959,2019,
         2078,2138,2198,2258,2318]
# for i in hourly_index:
#     print(acqu_time[i])

csv_file = pd.read_csv('NewportSalop_merged.csv', low_memory=False)
j = 32
# print(csv_file['Time'][j:j+40])
rainfall=csv_file[' Rainfall Total since 0900'][j:j+40].to_numpy()
rain_diff=np.diff(rainfall)
# rain_diff=rain_diff / np.max(rain_diff)

soil_temp_10=csv_file[' 10cm Soil Temperature'][j+1:j+40].to_numpy()
soil_temp_30=csv_file['30cm Soil Temperature'][j+1:j+40].to_numpy()
soil_temp_100=csv_file['100cm Soil Temperature'][j+1:j+40].to_numpy()
humidity=csv_file['Humidity'][j+1:j+40].to_numpy()

# for i in range(PSD_all_channel.shape[0]):
#     ave_psd=np.sum(PSD_all_channel[i, :, 52:90], axis=-1)
#     ave_psd=ave_psd / np.max(ave_psd)
#     ## smoothing over 1 hr window
#     hr_psd=[]
#     for ind in hourly_index:
#         st_ind = int(ind - 60)
#         ed_ind = st_ind + 60
#         psd_int=np.sum(ave_psd[st_ind:ed_ind])

#         hr_psd.append(psd_int)

#     hr_psd = hr_psd / max(hr_psd)
#     plt.figure(figsize = (12, 4), dpi = 200)   
#     plt.scatter(hourly_index, rain_diff/3.5, label='1 hr rain', color='g', marker='o', s=100)
#     plt.scatter(hourly_index, np.array(hr_psd), label='1 hr psd', color='b', marker='*', s=100)
#     plt.plot(hourly_index, np.array(hr_psd))
#     plt.plot(np.arange(ave_psd.shape[-1]), ave_psd, label='psd')
#     # plt.xticks(np.arange(0, data_plot.shape[1],100), xhrs[:data_plot.shape[1]:100], fontsize = 12)

#     plt.xlabel("ACF time (min)", fontsize = 16)
#     plt.ylabel("Sum of PSD", fontsize = 16)
#     plt.title('Channel ' + str(i+44), fontsize = 20)  
#     plt.ylim(-0.2, 1.2)
#     plt.legend()

ave_psd=np.mean(np.sum(PSD_all_time[0:44, :, 52:90], axis=-1) * (freq[1]-freq[0]), axis=0)
# ave_psd=ave_psd / np.max(ave_psd)
## smoothing over 1 hr window
hr_psd=[]
for ind in hourly_index:
    st_ind = int(ind - 60)
    ed_ind = st_ind + 60
    psd_int=np.sum(ave_psd[st_ind:ed_ind])

    hr_psd.append(psd_int)

# hr_psd = hr_psd / max(hr_psd)

xax=np.arange(0, ave_psd.shape[-1], 240)
tax=acqu_time[:ave_psd.shape[-1]:240]
hrax=[dt.strftime('%d %H:%M') for dt in tax]
plt.figure(figsize = (12, 4), dpi = 200)   
plt.scatter(hourly_index, rain_diff/3.5, label='1-hour rainfall / 3.5', color='g', marker='o', s=100)
plt.scatter(hourly_index, np.array(hr_psd), label='1-hour PSD', color='b', marker='*', s=100)
plt.plot(hourly_index, np.array(hr_psd))
plt.plot(np.arange(ave_psd.shape[-1]), ave_psd, label='1-minute PSD')
plt.xticks(xax, hrax, fontsize = 12)
plt.xlabel("Time", fontsize = 16)
plt.ylabel("Sum of PSD", fontsize = 16)
plt.title('Averaged PSD of the SW central channels ', fontsize = 20)  
plt.ylim(-0.2, 1.8)
plt.legend()


In [ ]:
with h5py.File('/fd1/QibinShi_data/England_farm/autocorr_25_50Hz.hdf5', 'r') as f:
    corr_all_time = f['autocorr'][:, :, 2500:2650, 0]

In [ ]:
win_st = int(0.012*samp_freq)

with h5py.File('/fd1/QibinShi_data/England_farm/autocorr_25_50Hz.hdf5', 'r') as f:
    corr_all_time = f['autocorr'][:, :, 2500:2650, 0]
    
for iloc in range(51):
    data_plot=corr_all_time[iloc].T
    x=np.arange(data_plot.shape[1])
    y=np.arange(data_plot.shape[0])
    xts=np.array(time_stamps)
    xhrs=np.round((xts - xts[0]).astype('float')/3600, 2)
    plt.figure(figsize = (10, 3), dpi = 200)
    for i in x[::5]:
        plt.plot(y/samp_freq, data_plot[:,i])

    plt.plot(y/samp_freq, np.mean(data_plot, axis=1), c='k', lw=9, alpha=0.3, label='mean of all')

    plt.legend()
    plt.title('channel '+ str(iloc+44))
    plt.xlabel('lag time (s)')
    plt.axvline(x=win_st/samp_freq, color='k', linestyle='--', label='1st reflection')

In [ ]:
time = y[win_st:] / samp_freq
stack_stretch=np.zeros((51, len(time)), dtype = np.float32)
for iloc in range(51):
    data_plot=corr_all_time[iloc].T
    x=np.arange(data_plot.shape[1])
    y=np.arange(data_plot.shape[0])
    trunc_acf = data_plot[win_st:, :]
    trunc_stk = np.mean(trunc_acf, axis=1)
    
    count = 0
    plt.figure(figsize = (10, 3), dpi = 200)
    for i in x[::5]:
        tmp=np.zeros_like(time)
        stretched, time_new, ratio, cc = stretch(trunc_stk, trunc_acf[:,i], time, max_ratio=2)
        if cc > 0.5:
            length=min(len(stretched), len(time))
            tmp[:length]=stretched[:length]
            stack_stretch[iloc, :] += tmp
            plt.plot(time, tmp)
            count +=1
        else:continue
    plt.plot(time, stack_stretch[iloc, :]/count, c='k', lw=9, alpha=0.5, label='mean of all')
    plt.title('channel '+ str(iloc+44)); plt.legend(); plt.xlabel('lag time (s)')

In [ ]:
freqmin=25
freqmax=freqmin*2
# Stretch again to get distribution
stretch_range = np.arange(1/2, 2, 0.01)
all_ratio = np.zeros((51, len(stretch_range), len(x)), dtype = np.float32)
all_cc = np.zeros((51, len(stretch_range), len(x)), dtype = np.float32)

xax=np.arange(0, x.shape[-1], 48)
tax=acqu_time[:2410:240]

for iloc in [8]:
    data_plot=corr_all_time[iloc].T
    trunc_acf = data_plot[win_st:, :]
    trunc_stk = stack_stretch[iloc, :]/count
    strecthed_acfs = np.zeros_like(trunc_acf)
    for i in x:
        ratios, ccs = stretch_distribution(trunc_stk, trunc_acf[:,i], time, max_ratio=2)
        all_cc[iloc, :, i] = ccs  
    plt.figure(figsize = (16, 4), dpi = 300)
    plt.pcolormesh(x, stretch_range-1, all_cc[iloc], shading='auto', vmin=0.2, vmax=1, cmap = 'hot')
    plt.xticks(xax, hrax, fontsize = 10)
    plt.xlabel("ACF time (hour)", fontsize = 16)
    plt.ylabel("dV / V", fontsize = 16)
    plt.ylim(-0.5, 0.6)
    plt.title('channel '+ str(iloc+44)+ '_' + str(freqmin)+'_'+str(freqmax)+'Hz', fontsize = 20)
    bar = plt.colorbar()
    bar.set_label('Cross-correlation Coefficient', fontsize = 15)
    
    axcopy = plt.twinx()
#     axcopy.scatter(np.array(hourly_index)/5, soil_temp_10-7.5, label='T_10cm', color='w', marker='^')
#     axcopy.scatter(np.array(hourly_index)/5, (soil_temp_30-7.5), label='T_30cm', color='w', marker='s')
#     axcopy.scatter(np.array(hourly_index)/5, (soil_temp_100-7.5), label='T_100cm', color='w', marker='v')
    axcopy.scatter(np.array(hourly_index)/5, (110-humidity)/250, label='(110-RH) / 250', color='cyan', marker='P', s=100)
#     axcopy.scatter(np.array(hourly_index)/5, rain_diff/3.5, label='1-hour rainfall', color='g', marker='o', s=100)
#     axcopy.scatter(np.array(hourly_index)/5, np.array(hr_psd), label='1-hour PSD', color='b', marker='*', s=100)
#     axcopy.plot(np.array(hourly_index)/5, np.array(hr_psd))
    axcopy.plot(np.arange(ave_psd.shape[-1])/5, ave_psd, label='1-minute PSD', color='w')
    axcopy.legend(fontsize = 7)
    axcopy.set_ylim(-0.02,0.2)

In [ ]:
# Stretch again to get distribution
stretch_range = np.arange(1/2, 2, 0.01)
all_ratio = np.zeros((51, len(stretch_range), len(x)), dtype = np.float32)
all_cc = np.zeros((51, len(stretch_range), len(x)), dtype = np.float32)

xax=np.arange(0, x.shape[-1], 48)
tax=acqu_time[:2410:240]

# for iloc in [18,30]:
for iloc in range(51):
    data_plot=corr_all_time[iloc].T
    trunc_acf = data_plot[win_st:, :]
    trunc_stk = stack_stretch[iloc, :]/count
    strecthed_acfs = np.zeros_like(trunc_acf)
    for i in x:
        ratios, ccs = stretch_distribution(trunc_stk, trunc_acf[:,i], time, max_ratio=2)
        all_cc[iloc, :, i] = ccs 
        
    ######################
    ## FK filter
    # %% Use cone-filter
    filt_data, filtered_fk, fk2d = fk_filter_2cones(all_cc[iloc].T, w1=0.03, w2=0.005, cone1=True, cone2=False)

#     plt.figure(figsize=(18, 5))
#     plt.imshow(filt_data.real.T, aspect = 'auto', cmap = 'hot', vmax = 0.2, vmin = 1, origin='lower')
#     plt.title('Mask filter after 2D Fourier Transform')
#     plt.colorbar()

#     ## verify the FK mask
#     plt.figure(figsize=(18, 5), dpi=300)
#     plt.imshow(np.log10(np.abs(filtered_fk)), aspect='auto', cmap='RdBu', origin='lower')
#     plt.colorbar()
#     plt.figure(figsize=(18, 5), dpi=300)
#     plt.imshow(np.log10(np.abs(fk2d)), aspect='auto', cmap='RdBu', origin='lower')
#     plt.colorbar()
    #######################
    image = filt_data.real.T[:,:,np.newaxis]*200
    image = image - np.nanmin(image)
    denoised_image = cv2.medianBlur(image.astype(np.uint8), 7)/256
    plt.figure(figsize=(18, 5))
    plt.pcolormesh(x, stretch_range-1, denoised_image, shading='auto', vmin=0.3, vmax=0.8, cmap = 'hot')
    plt.title('median filter in 2D_iloc' + str(iloc))
    plt.xticks(xax, hrax, fontsize = 10)
    bar=plt.colorbar()
    bar.set_label('Cross-correlation Coefficient', fontsize = 15)
    plt.xlabel("time", fontsize = 16)
    plt.ylabel("dV / V", fontsize = 16)
    #######################
#     kernel_size_x = 2
#     denoised_image_x = np.zeros_like(denoised_image)
#     for i in range(image.shape[1]):
#         denoised_image_x[:, i] = np.median(image[:, i - kernel_size_x:i + kernel_size_x + 1, 0], axis=1)
   
#     plt.figure(figsize=(18, 5))
#     plt.imshow(denoised_image_x, aspect = 'auto', cmap = 'hot', vmax=200, vmin=80, origin='lower')
#     plt.title('median filter along X')
#     plt.colorbar()
    #######################
    plt.figure(figsize=(18, 5))
    dvv_ind = np.argmax(denoised_image, axis=0)
    dvv = np.zeros(len(dvv_ind), dtype=np.float32)
    for i in range(len(dvv_ind)):
        dvv[i]= stretch_range[dvv_ind[i]]
    cc_dvv = np.nanmax(denoised_image, axis=0)
    plt.scatter(x, dvv-np.mean(dvv), cmap='viridis', c=cc_dvv, s=40, marker='o')
    plt.xticks(xax, hrax, fontsize = 10)
    plt.title('dVV after denoising')
    plt.ylim(-0.6, 0.6)
    bar=plt.colorbar()
    bar.set_label('Cross-correlation Coefficient', fontsize = 15)
    plt.xlabel("time", fontsize = 16)
    plt.ylabel("dV / V", fontsize = 16)
    
    # Save all stretch ratio and CC
    with h5py.File('dvv_stretch_cc_25_50Hz.hdf5', 'r') as f:
        ratio_all = f['ratio'][:]
        cc_all = f['cc'][:]
    plt.figure(figsize = (18, 5))
    plt.scatter(x, ratio_all[iloc]-np.mean(ratio_all[iloc]), cmap='viridis', c=np.array(cc_all[iloc]), s=40, marker='o')
    plt.xticks(xax, hrax, fontsize = 10)
    plt.title('dVV raw')
    plt.ylim(-0.6, 0.6)
    bar=plt.colorbar()
    bar.set_label('Cross-correlation Coefficient', fontsize = 15)
    plt.xlabel("time", fontsize = 16)
    plt.ylabel("dV / V", fontsize = 16)


   

In [ ]:
# Stretch again to get distribution
stretch_range = np.arange(1/2, 2, 0.01)
all_ratio = np.zeros((51, len(stretch_range), len(x)), dtype = np.float32)
all_cc = np.zeros((51, len(stretch_range), len(x)), dtype = np.float32)

xax=np.arange(0, x.shape[-1], 48)
tax=acqu_time[:2410:240]

for iloc in [8, 18, 30]:
# for iloc in range(51):
    data_plot=corr_all_time[iloc].T
    trunc_acf = data_plot[win_st:, :]
    trunc_stk = stack_stretch[iloc, :]/count
    strecthed_acfs = np.zeros_like(trunc_acf)
    for i in x:
        ratios, ccs = stretch_distribution(trunc_stk, trunc_acf[:,i], time, max_ratio=2)
        all_cc[iloc, :, i] = ccs 
        
    ######################
    ## FK filter
    # %% Use cone-filter
    filt_data, filtered_fk, fk2d = fk_filter_2cones(all_cc[iloc].T, w1=0.015, w2=0.005, cone1=True, cone2=False)

    plt.figure(figsize=(18, 5))
    plt.pcolormesh(x, stretch_range-1, filt_data.real.T, shading='auto', vmin=0.2, vmax=1, cmap = 'hot')
    plt.title('Mask filter after 2D Fourier Transform')
    plt.xticks(xax, hrax, fontsize = 10)
    bar=plt.colorbar()
    bar.set_label('Cross-correlation Coefficient', fontsize = 15)
    plt.xlabel("time", fontsize = 16)
    plt.ylabel("dV / V", fontsize = 16)

    ## verify the FK mask
    plt.figure(figsize=(18, 5), dpi=300)
    plt.imshow(np.log10(np.abs(filtered_fk)), aspect='auto', cmap='RdBu', origin='lower')
    bar=plt.colorbar()
    bar.set_label('Relative power', fontsize = 15)
    plt.figure(figsize=(18, 5), dpi=300)
    plt.imshow(np.log10(np.abs(fk2d)), aspect='auto', cmap='RdBu', origin='lower')
    bar=plt.colorbar()
    bar.set_label('Relative power', fontsize = 15)
    #######################
    image = filt_data.real.T[:,:,np.newaxis]*200
    image = image - np.nanmin(image)
    denoised_image = cv2.medianBlur(image.astype(np.uint8), 7)/256
    plt.figure(figsize=(18, 5))
    plt.pcolormesh(x, stretch_range-1, denoised_image, shading='auto', vmin=0.3, vmax=0.8, cmap = 'hot')
    plt.title('median filter in 2D')
    plt.xticks(xax, hrax, fontsize = 10)
    bar=plt.colorbar()
    bar.set_label('Cross-correlation Coefficient', fontsize = 15)
    plt.xlabel("time", fontsize = 16)
    plt.ylabel("dV / V", fontsize = 16)
    #######################
#     kernel_size_x = 2
#     denoised_image_x = np.zeros_like(denoised_image)
#     for i in range(image.shape[1]):
#         denoised_image_x[:, i] = np.median(image[:, i - kernel_size_x:i + kernel_size_x + 1, 0], axis=1)
   
#     plt.figure(figsize=(18, 5))
#     plt.imshow(denoised_image_x, aspect = 'auto', cmap = 'hot', vmax=200, vmin=80, origin='lower')
#     plt.title('median filter along X')
#     plt.colorbar()
    #######################
    plt.figure(figsize=(18, 5))
    dvv_ind = np.argmax(denoised_image, axis=0)
    dvv = np.zeros(len(dvv_ind), dtype=np.float32)
    for i in range(len(dvv_ind)):
        dvv[i]= stretch_range[dvv_ind[i]]
    cc_dvv = np.nanmax(denoised_image, axis=0)
    plt.scatter(x, dvv-np.mean(dvv), cmap='viridis', c=cc_dvv, s=40, marker='o')
    plt.xticks(xax, hrax, fontsize = 10)
    plt.title('dVV after denoising')
    plt.ylim(-0.6, 0.6)
    bar=plt.colorbar()
    bar.set_label('Cross-correlation Coefficient', fontsize = 15)
    plt.xlabel("time", fontsize = 16)
    plt.ylabel("dV / V", fontsize = 16)
    
    # Save all stretch ratio and CC
    with h5py.File('dvv_stretch_cc_25_50Hz.hdf5', 'r') as f:
        ratio_all = f['ratio'][:]
        cc_all = f['cc'][:]
    plt.figure(figsize = (18, 5))
    plt.scatter(x, ratio_all[iloc]-np.mean(ratio_all[iloc]), cmap='viridis', c=np.array(cc_all[iloc]), s=40, marker='o')
    plt.xticks(xax, hrax, fontsize = 10)
    plt.title('dVV raw')
    plt.ylim(-0.6, 0.6)
    bar=plt.colorbar()
    bar.set_label('Cross-correlation Coefficient', fontsize = 15)
    plt.xlabel("time", fontsize = 16)
    plt.ylabel("dV / V", fontsize = 16)


In [ ]:
for i in range(len(dvv_ind)):
    dvv[i]= stretch_range[dvv_ind[i]]
    
stretch_range[63]